## Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import IPython.display as ipd
import librosa
import librosa.display

## Defining Constants

In [2]:
path = "/Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Data/"
Model_path = "/Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/"
audio_file = "UrbanSound8K/audio/"
audio_dataset_path = path+audio_file
metadata = "UrbanSound8K/metadata/UrbanSound8K.csv"

num_epochs = 200
num_batch_size = 32

In [3]:
meta_data = pd.read_csv(path+metadata)

meta_data.shape

(8732, 8)

In [4]:
meta_data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


## Creating Features

In [5]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, 
                                      res_type = "kaiser_fast")
    mfccs_features = librosa.feature.mfcc(y = audio, 
                                          sr = sample_rate,
                             # n_mfcc is a hyperparameter, we can try different different values such as 30, 50, 60 etc.
                                          n_mfcc = 40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)
    
    return (mfccs_scaled_features)

## Iterating through every audio file for feature extraction using MFCC

In [6]:
extracted_features = []

for index_num, row in tqdm(meta_data.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), "fold"+str(row["fold"])+"/", str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])

3554it [02:01, 29.62it/s]/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8323it [04:34, 44.10it/s]/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [04:46, 30.43it/s]


## Converting the extracted features to DataFrame

In [7]:
extracted_features_df = pd.DataFrame(data = extracted_features, columns = ["feature", "class"])

extracted_features_df.shape

(8732, 2)

In [8]:
extracted_features_df.head()

,feature,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.520657, 52.00812, ...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036358, ...",children_playing
4,"[-446.60352, 113.68541, -52.402206, 60.302044,...",children_playing


## Splitting the dataset into independent and dependent features

In [9]:
X = np.array(extracted_features_df["feature"].values.tolist())
y = np.array(extracted_features_df["class"].values.tolist())

print(X.shape, y.shape)

(8732, 40) (8732,)


In [11]:
y

array(['dog_bark', 'children_playing', 'children_playing', ...,
       'car_horn', 'car_horn', 'car_horn'], dtype='<U16')

## Encoding the target

In [13]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

y = to_categorical(labelencoder.fit_transform(y))

y.shape

(8732, 10)

In [14]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

## Splitting the dataset into Train and Test

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6985, 40) (1747, 40) (6985, 10) (1747, 10)


## Finding number of classes in Target variable

In [16]:
num_labels = y.shape[1]

In [17]:
num_labels

10

## Model Creation

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## ANN Model

In [19]:
model = Sequential()
#First layer
model.add(Dense(units = 100, input_shape = (40,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Second layer
model.add(Dense(units = 200))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Third layer
model.add(Dense(units = 100))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Final layer
model.add(Dense(num_labels))
model.add(Activation("softmax"))

## Model Summary

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

## Model Compilation

In [21]:
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

## Model Training

In [41]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

checkpointer = ModelCheckpoint(filepath = Model_path+"audio_classification.hdf5", 
                               verbose =1, 
                               save_best_only = True)

start = datetime.now()

model.fit(X_train, y_train, 
          batch_size = num_batch_size, 
          epochs = num_epochs, 
          validation_data = (X_test, y_test),
          callbacks = [checkpointer],
          verbose = 1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
210/219 [===========================>..] - ETA: 0s - loss: 0.9006 - accuracy: 0.7060
Epoch 1: val_loss improved from inf to 0.65800, saving model to /Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9012 - accuracy: 0.7059 - val_loss: 0.6580 - val_accuracy: 0.7945
Epoch 2/200
217/219 [============================>.] - ETA: 0s - loss: 0.9001 - accuracy: 0.7092
Epoch 2: val_loss improved from 0.65800 to 0.65352, saving model to /Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9008 - accuracy: 0.7089 - val_loss: 0.6535 - val_accuracy: 0.7979
Epoch 3/200
217/219 [============================>.] - ETA: 0s - loss: 0.8828 - accuracy: 0.7088
Epoch 3: val_loss did not improve from 0.65352
219/219 [=====

Epoch 28/200
218/219 [============================>.] - ETA: 0s - loss: 0.8819 - accuracy: 0.7057
Epoch 28: val_loss improved from 0.64179 to 0.63455, saving model to /Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.8822 - accuracy: 0.7057 - val_loss: 0.6346 - val_accuracy: 0.8117
Epoch 29/200
165/219 [=====================>........] - ETA: 0s - loss: 0.8752 - accuracy: 0.7081
Epoch 29: val_loss did not improve from 0.63455
219/219 [==============================] - 0s 1ms/step - loss: 0.8788 - accuracy: 0.7068 - val_loss: 0.6578 - val_accuracy: 0.8060
Epoch 30/200
165/219 [=====================>........] - ETA: 0s - loss: 0.8658 - accuracy: 0.7089
Epoch 30: val_loss did not improve from 0.63455
219/219 [==============================] - 0s 1ms/step - loss: 0.8726 - accuracy: 0.7079 - val_loss: 0.6360 - val_accuracy: 0.8071
Epoch 31/200
165/219 

Epoch 56/200
218/219 [============================>.] - ETA: 0s - loss: 0.8855 - accuracy: 0.7169
Epoch 56: val_loss did not improve from 0.62920
219/219 [==============================] - 0s 1ms/step - loss: 0.8856 - accuracy: 0.7168 - val_loss: 0.6614 - val_accuracy: 0.7916
Epoch 57/200
219/219 [==============================] - ETA: 0s - loss: 0.8906 - accuracy: 0.7148
Epoch 57: val_loss did not improve from 0.62920
219/219 [==============================] - 0s 1ms/step - loss: 0.8906 - accuracy: 0.7148 - val_loss: 0.6399 - val_accuracy: 0.8037
Epoch 58/200
166/219 [=====================>........] - ETA: 0s - loss: 0.8631 - accuracy: 0.7212
Epoch 58: val_loss did not improve from 0.62920
219/219 [==============================] - 0s 1ms/step - loss: 0.8772 - accuracy: 0.7152 - val_loss: 0.6450 - val_accuracy: 0.8117
Epoch 59/200
213/219 [============================>.] - ETA: 0s - loss: 0.8888 - accuracy: 0.7077
Epoch 59: val_loss did not improve from 0.62920
219/219 [==============

215/219 [============================>.] - ETA: 0s - loss: 0.8666 - accuracy: 0.7158
Epoch 85: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8671 - accuracy: 0.7162 - val_loss: 0.6556 - val_accuracy: 0.7922
Epoch 86/200
166/219 [=====================>........] - ETA: 0s - loss: 0.8638 - accuracy: 0.7176
Epoch 86: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8704 - accuracy: 0.7168 - val_loss: 0.6453 - val_accuracy: 0.8042
Epoch 87/200
166/219 [=====================>........] - ETA: 0s - loss: 0.8731 - accuracy: 0.7052
Epoch 87: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8533 - accuracy: 0.7128 - val_loss: 0.6396 - val_accuracy: 0.8025
Epoch 88/200
219/219 [==============================] - ETA: 0s - loss: 0.8537 - accuracy: 0.7190
Epoch 88: val_loss did not improve from 0.62404
219/219 [===========================

219/219 [==============================] - 0s 1ms/step - loss: 0.8674 - accuracy: 0.7141 - val_loss: 0.6355 - val_accuracy: 0.8151
Epoch 115/200
219/219 [==============================] - ETA: 0s - loss: 0.8465 - accuracy: 0.7247
Epoch 115: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8465 - accuracy: 0.7247 - val_loss: 0.6315 - val_accuracy: 0.8111
Epoch 116/200
168/219 [======================>.......] - ETA: 0s - loss: 0.8414 - accuracy: 0.7208
Epoch 116: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8440 - accuracy: 0.7214 - val_loss: 0.6332 - val_accuracy: 0.8088
Epoch 117/200
166/219 [=====================>........] - ETA: 0s - loss: 0.8276 - accuracy: 0.7218
Epoch 117: val_loss did not improve from 0.62404
219/219 [==============================] - 0s 1ms/step - loss: 0.8439 - accuracy: 0.7167 - val_loss: 0.6376 - val_accuracy: 0.8180
Epoch 118/200
218/219 [=========

219/219 [==============================] - 0s 1ms/step - loss: 0.8346 - accuracy: 0.7260 - val_loss: 0.6125 - val_accuracy: 0.8174
Epoch 144/200
217/219 [============================>.] - ETA: 0s - loss: 0.8501 - accuracy: 0.7247
Epoch 144: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8494 - accuracy: 0.7247 - val_loss: 0.6227 - val_accuracy: 0.8163
Epoch 145/200
166/219 [=====================>........] - ETA: 0s - loss: 0.8299 - accuracy: 0.7272
Epoch 145: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8318 - accuracy: 0.7254 - val_loss: 0.6344 - val_accuracy: 0.8145
Epoch 146/200
165/219 [=====================>........] - ETA: 0s - loss: 0.8381 - accuracy: 0.7165
Epoch 146: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8505 - accuracy: 0.7221 - val_loss: 0.6513 - val_accuracy: 0.8042
Epoch 147/200
164/219 [=========

Epoch 173/200
212/219 [============================>.] - ETA: 0s - loss: 0.8516 - accuracy: 0.7245
Epoch 173: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8513 - accuracy: 0.7246 - val_loss: 0.6130 - val_accuracy: 0.8105
Epoch 174/200
219/219 [==============================] - ETA: 0s - loss: 0.8307 - accuracy: 0.7218
Epoch 174: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8307 - accuracy: 0.7218 - val_loss: 0.6136 - val_accuracy: 0.8037
Epoch 175/200
216/219 [============================>.] - ETA: 0s - loss: 0.8129 - accuracy: 0.7348
Epoch 175: val_loss did not improve from 0.60655
219/219 [==============================] - 0s 1ms/step - loss: 0.8147 - accuracy: 0.7344 - val_loss: 0.6255 - val_accuracy: 0.8145
Epoch 176/200
219/219 [==============================] - ETA: 0s - loss: 0.8482 - accuracy: 0.7201
Epoch 176: val_loss did not improve from 0.60655
219/219 [======

In [42]:
accuracy = model.evaluate(X_test, y_test, verbose = 0)
print(accuracy[1])

0.8191184997558594


## Working with Test Data

In [37]:
test_filename = "car_noise.wav"

# Feature extraction 
audio, sample_rate = librosa.load(path+test_filename, res_type = "kaiser_fast")
mfccs_features = librosa.feature.mfcc(y = audio, 
                                      sr = sample_rate, 
                                      n_mfcc = 40)

# Normalizing extracted features
mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)

print(mfccs_scaled_features)

# Converting normalized features into 1d array
mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)

print("\n", mfccs_scaled_features)
print("\n", mfccs_scaled_features.shape)

[-196.822       113.993126    -13.813408      0.40220982  -20.145586
   -4.7625513   -40.68413       4.171539    -18.973984     -3.0413852
  -19.736597      7.505515    -21.69197      -4.053084    -20.116352
    4.8145556   -14.817319      0.9286656   -14.061541      8.725959
   -6.6515527     6.45562      -4.6819024     0.5490028    -9.6971445
   -0.34578854   -8.68798      -0.6733791    -6.8771744     5.059755
  -10.099091     -0.2596309    -2.8073726     4.4415655    -7.439074
   -4.481123     -1.9385126     3.842505     -5.947936     -0.98801756]

 [[-196.822       113.993126    -13.813408      0.40220982  -20.145586
    -4.7625513   -40.68413       4.171539    -18.973984     -3.0413852
   -19.736597      7.505515    -21.69197      -4.053084    -20.116352
     4.8145556   -14.817319      0.9286656   -14.061541      8.725959
    -6.6515527     6.45562      -4.6819024     0.5490028    -9.6971445
    -0.34578854   -8.68798      -0.6733791    -6.8771744     5.059755
   -10.099091     -

## Predicting the test data

In [38]:
predicted_label = np.argmax(model.predict(mfccs_scaled_features),axis=1)

print(predicted_label)

1/1 [==============================] - 0s 46ms/step
[9]


## Finding the class of the predicted value

In [40]:
prediction_class = labelencoder.inverse_transform(predicted_label)

print(prediction_class[0])

street_music
